In [1]:
import sys
import os
from timeit import default_timer as timer
import torch
from transformers import RobertaTokenizer, RobertaModel, RobertaForMaskedLM, RobertaConfig

In [2]:
# Make sure we're in the transformers directory with fine-tuned model output.
os.chdir('/Users/crystal.butler/Documents/Code_Projects/RoBERTa_Embeddings/RoBERTa_embeddings/')
os.getcwd()

'/Users/crystal.butler/Documents/Code_Projects/RoBERTa_Embeddings/RoBERTa_embeddings'

In [3]:
# Adapted from the tutorial at https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
# and Transformers documentation: https://huggingface.co/transformers/model_doc/roberta.html#robertaformaskedlm

In [37]:
tokenizer = RobertaTokenizer.from_pretrained('/Users/crystal.butler/Documents/Code_Projects/RoBERTa_Embeddings/RoBERTa_embeddings/data/output_wiki-103_filtered')
config = RobertaConfig.from_pretrained('/Users/crystal.butler/Documents/Code_Projects/RoBERTa_Embeddings/RoBERTa_embeddings/data/output_wiki-103_filtered')
model = RobertaForMaskedLM.from_pretrained('/Users/crystal.butler/Documents/Code_Projects/RoBERTa_Embeddings/RoBERTa_embeddings/data/output_wiki-103_filtered', config=config)
model.eval()

context_file = "/Users/crystal.butler/Documents/Code_Projects/RoBERTa_Embeddings/RoBERTa_embeddings/data/wiki.test.raw.out"
output_file = '/Users/crystal.butler/Documents/Code_Projects/RoBERTa_Embeddings/RoBERTa_embeddings/data/roberta_test.txt'
count_file = '/Users/crystal.butler/Documents/Code_Projects/RoBERTa_Embeddings/RoBERTa_embeddings/data/roberta_test_counts.txt'
vocab_file = '/Users/crystal.butler/Documents/Code_Projects/RoBERTa_Embeddings/RoBERTa_embeddings/data/vocab_checked.txt'
vocab = make_vocab(vocab_file)
FEATURE_COUNT = 768
MAX_LINES = 2000

In [43]:
# Process vocabulary words in the outer loop.
for v in vocab:
    start = timer()
    with open(context_file, 'r') as lines:
        v_sum = torch.zeros([1, FEATURE_COUNT])
        v_tokens = tokenize_text(v, tokenizer)
        print_tokenized_text(v_tokens, tokenizer)
        count_sentence = 0
        count_tensor = 0
        
        # Process all lines in the context file in the inner loop.
        for line in lines:
            # Check for this vocab word in this line; if found, split the line into individual sentences.
            if v in line.lower().split():
                for sentence in line.split('.'):
                    if v in sentence.lower():
                        line = sentence
                        count_sentence += 1
                        print(f'\nInstance {count_sentence} of {tokenizer.decode(v_tokens[1:-1]).strip()}.')
                        break  # We'll take the first instance of the word and discard the rest of the line.
                # Split the new sentence-based line into tokens.
                line_tokens = tokenize_text(line, tokenizer)               
                # Get the indices of the line at which our vocabulary word tokens are located.
                indices = get_vocab_indices(v_tokens, line_tokens, tokenizer)                             

                # If the vocabulary word was found, process the containing line.
                if indices:
                    # Get the feature vectors for all tokens in the line/sentence.
                    token_embeddings = create_token_embeddings(line_tokens)
                    # Select a method for extracting specific layers of the model.
                    token_vecs_layer = get_layer_token_vecs(token_embeddings, 12)
                    # Sum the individual token contextual embeddings for the whole vocab word, for this line.
                    tensor_layer = torch.zeros([1, FEATURE_COUNT])
                    for i in range(len(indices)):
                        preview_token_embedding(v_tokens, token_vecs_layer, i, indices, tokenizer)
                        tensor_layer += token_vecs_layer[indices[i]]
                    # If our vocab word is broken into more than one token, we need to get the mean of the token embeddings.
                    tensor_layer /= len(indices)

                    # Add the embedding distilled from this line to the sum of embeddings for all lines.
                    v_sum += tensor_layer
                    count_tensor += 1
                    print(f'Grand sum of {count_tensor} tensor sets is: {v_sum[0][:5].tolist()}')

            # Stop processing lines once we've found 2000 instances of our vocab word.
            if count_tensor >= MAX_LINES:
                break
        
        # We're done processing all lines of 512 tokens or less containing our vocab word.
        # Get the mean embedding for the word.
        v_mean = v_sum / count_tensor
        print(f'Mean of {count_tensor} tensors is: {v_mean[0][:5]} (first 5 of {len(v_mean[0])} features in tensor)')
        write_embedding(output_file, v, v_mean)
        write_line_count(count_file, v, count_tensor)
    end = timer()
    print(f'Run time for {v} was {end - start} seconds.')


There are 1 tokens in tokenized text:
aback
Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved the embedding for aback.
Saved the count of sentences used to create aback embedding
Run time for aback was 0.030610576999606565 seconds.

There are 2 tokens in tokenized text:
ab
ashed
Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved the embedding for abashed.
Saved the count of sentences used to create abashed embedding
Run time for abashed was 0.030524277000040456 seconds.

There are 1 tokens in tokenized text:
abhor
Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved the embedding for abhor.
Saved the count of sentences used to create abhor embedding
Run time for abhor was 0.028823298999668623 seconds.

There are 2 tokens in tokenized text:
abhor
red
Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved t

Size of token embeddings is torch.Size([37, 13, 768])
Shape of summed layers is: 37 x 768
accepted at index 22:  [-0.005839262157678604, 0.0065529122948646545, 0.007053704932332039, 0.2620042860507965, 0.30201423168182373]
Grand sum of 5 tensor sets is: [0.3896675705909729, 0.37197333574295044, -0.18983963131904602, 1.792115330696106, 1.7109131813049316]

Instance 6 of accepted.
Size of token embeddings is torch.Size([21, 13, 768])
Shape of summed layers is: 21 x 768
accepted at index 19:  [0.055956169962882996, 0.10654482245445251, -0.1882839798927307, 0.127366304397583, 0.40761083364486694]
Grand sum of 6 tensor sets is: [0.4456237554550171, 0.47851815819740295, -0.37812361121177673, 1.919481635093689, 2.1185240745544434]

Instance 7 of accepted.
Size of token embeddings is torch.Size([28, 13, 768])
Shape of summed layers is: 28 x 768
accepted at index 3:  [0.04771412909030914, -0.06445986032485962, -0.07482258975505829, 0.20206670463085175, 0.9173025488853455]
Grand sum of 7 tensor 

Size of token embeddings is torch.Size([27, 13, 768])
Shape of summed layers is: 27 x 768
accomplished at index 2:  [0.13993659615516663, 0.07892926037311554, 0.40447723865509033, -0.07857934385538101, 0.22581759095191956]
Grand sum of 2 tensor sets is: [0.09603787958621979, 0.07234682142734528, 0.6336098909378052, -0.2386748194694519, 0.3055179715156555]

Instance 3 of accomplished.
Size of token embeddings is torch.Size([19, 13, 768])
Shape of summed layers is: 19 x 768
accomplished at index 11:  [0.040347304195165634, 0.3676615357398987, 0.15343107283115387, -0.23540496826171875, -0.05082383751869202]
Grand sum of 3 tensor sets is: [0.13638518750667572, 0.44000834226608276, 0.7870409488677979, -0.47407978773117065, 0.2546941339969635]
Mean of 3 tensors is: tensor([ 0.0455,  0.1467,  0.2623, -0.1580,  0.0849]) (first 5 of 768 features in tensor)
Saved the embedding for accomplished.
Saved the count of sentences used to create accomplished embedding
Run time for accomplished was 0.190

Size of token embeddings is torch.Size([37, 13, 768])
Shape of summed layers is: 37 x 768
active at index 29:  [0.04733189195394516, -0.12996122241020203, 0.03889086842536926, -0.04028075188398361, 0.663524866104126]
Grand sum of 8 tensor sets is: [0.5281392335891724, 0.3307981491088867, 0.7075533866882324, 2.372034788131714, -2.9380874633789062]

Instance 9 of active.
Size of token embeddings is torch.Size([20, 13, 768])
Shape of summed layers is: 20 x 768
active at index 17:  [-0.12019650638103485, -0.2308669537305832, 0.02476239576935768, 0.20746958255767822, -0.7091408371925354]
Grand sum of 9 tensor sets is: [0.4079427123069763, 0.09993119537830353, 0.7323157787322998, 2.5795044898986816, -3.647228240966797]

Instance 10 of active.
Size of token embeddings is torch.Size([35, 13, 768])
Shape of summed layers is: 35 x 768
active at index 5:  [0.005576286464929581, 0.0764036551117897, 0.0006837770342826843, 0.5913102626800537, -0.4659793972969055]
Grand sum of 10 tensor sets is: [0.4

Mean of 3 tensors is: tensor([-0.0403, -0.1685,  0.2886, -0.4222, -0.2433]) (first 5 of 768 features in tensor)
Saved the embedding for admiration.
Saved the count of sentences used to create admiration embedding
Run time for admiration was 0.18663836900032038 seconds.

There are 1 tokens in tokenized text:
admit
Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved the embedding for admit.
Saved the count of sentences used to create admit embedding
Run time for admit was 0.026911353999821586 seconds.

There are 2 tokens in tokenized text:
ad
oration
Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved the embedding for adoration.
Saved the count of sentences used to create adoration embedding
Run time for adoration was 0.026020743000117363 seconds.

There are 2 tokens in tokenized text:
ad
oring
Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved the e

...

Size of token embeddings is torch.Size([43, 13, 768])
Shape of summed layers is: 43 x 768
certain at index 23:  [0.08014874160289764, 0.20246955752372742, 0.16360749304294586, -0.15099219977855682, 0.542007565498352]
Grand sum of 3 tensor sets is: [0.4104467034339905, 0.7836649417877197, 0.23979482054710388, 0.021361589431762695, 0.9238044619560242]

Instance 4 of certain.
Size of token embeddings is torch.Size([76, 13, 768])
Shape of summed layers is: 76 x 768
certain at index 74:  [0.329013854265213, 0.11649209260940552, 0.011987544596195221, 0.2407076507806778, -0.110133096575737]
Grand sum of 4 tensor sets is: [0.7394605875015259, 0.9001570343971252, 0.2517823576927185, 0.2620692253112793, 0.813671350479126]

Instance 5 of certain.
Size of token embeddings is torch.Size([53, 13, 768])
Shape of summed layers is: 53 x 768
certain at index 40:  [0.12302755564451218, 0.6831939220428467, 0.35198304057121277, -0.26052817702293396, 1.6595549583435059]
Grand sum of 5 tensor sets is: [0.862

Size of token embeddings is torch.Size([39, 13, 768])
Shape of summed layers is: 39 x 768
challenge at index 26:  [-0.1769009828567505, 0.206762433052063, -0.03942039608955383, -0.030211595818400383, 0.04174128174781799]
Grand sum of 8 tensor sets is: [-0.9991161823272705, -0.47883135080337524, 0.11006081104278564, 0.701418936252594, 3.206609010696411]

Instance 12 of challenge.
Size of token embeddings is torch.Size([21, 13, 768])
Shape of summed layers is: 21 x 768
challenge at index 6:  [0.008087854832410812, -0.04668854922056198, 0.0518520325422287, -0.08244245499372482, 0.26313671469688416]
Grand sum of 9 tensor sets is: [-0.9910283088684082, -0.5255199074745178, 0.16191284358501434, 0.6189764738082886, 3.469745635986328]

Instance 13 of challenge.
Size of token embeddings is torch.Size([36, 13, 768])
Shape of summed layers is: 36 x 768
challenge at index 26:  [-0.0661962479352951, -0.19050191342830658, -0.06354019045829773, -0.09178368747234344, 0.16393277049064636]
Grand sum of 

Size of token embeddings is torch.Size([57, 13, 768])
Shape of summed layers is: 57 x 768
charged at index 17:  [-0.04074686020612717, 0.06542021036148071, -0.07458929717540741, 0.27974164485931396, 0.06783347576856613]
Grand sum of 4 tensor sets is: [-0.15246132016181946, 0.3552277088165283, -0.14645852148532867, 0.6768467426300049, 1.1177630424499512]

Instance 5 of charged.
Size of token embeddings is torch.Size([22, 13, 768])
Shape of summed layers is: 22 x 768
charged at index 18:  [-0.12716367840766907, 0.0030889660120010376, -0.14080283045768738, -0.0530497282743454, 0.2771533131599426]
Grand sum of 5 tensor sets is: [-0.2796249985694885, 0.35831665992736816, -0.28726136684417725, 0.6237969994544983, 1.394916296005249]

Instance 6 of charged.
Size of token embeddings is torch.Size([22, 13, 768])
Shape of summed layers is: 22 x 768
charged at index 2:  [-0.14574426412582397, 0.1371181309223175, -0.048487864434719086, 0.0754559189081192, 0.24070662260055542]
Grand sum of 6 tensor 

Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved the embedding for chortling.
Saved the count of sentences used to create chortling embedding
Run time for chortling was 0.026725345999693673 seconds.

There are 1 tokens in tokenized text:
chuckle
Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved the embedding for chuckle.
Saved the count of sentences used to create chuckle embedding
Run time for chuckle was 0.029205514000750554 seconds.

There are 2 tokens in tokenized text:
chuck
ling
Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved the embedding for chuckling.
Saved the count of sentences used to create chuckling embedding
Run time for chuckling was 0.027583215000049677 seconds.

There are 3 tokens in tokenized text:
ch
ur
lish
Mean of 0 tensors is: tensor([nan, nan, nan, nan, nan]) (first 5 of 768 features in tensor)
Saved the embedding for

KeyboardInterrupt: 

In [12]:
def make_vocab(vocab_file):
    """Convert a file of newline separated words into a Python list and return it."""
    vocab = []
    with open(vocab_file, 'r') as v:
        vocab = v.read().splitlines()
    return vocab

In [20]:
def tokenize_text(text, tokenizer):
    """Break the input text into tokens the model can use, and return them.
    Use max_length to avoid overflowing the maximum sequence length for the model."""
    tokenized_text = tokenizer.encode(text, add_special_tokens=True, max_length=512)
    return tokenized_text

In [18]:
def print_tokenized_text(tokens, tokenizer):
    """Print the number of tokens in some tokenized text, not counting the leading and trailing separators.
    Print each token without any leading or trailing whitespace."""
    print(f'\nThere are {len(tokens) - 2} tokens in tokenized text:')
    for t in tokens[1:-1]:
        print(tokenizer.decode(t).strip())

In [21]:
def get_vocab_indices(v_tokens, line_tokens, tokenizer):
    """Search a line for all tokens of a vocabulary word, and return the indices of their locations."""
    indices = []              
    for t in v_tokens[1:-1]:
        for i, token_str in enumerate(line_tokens):
            if tokenizer.decode(token_str).strip() == tokenizer.decode(t).strip():
                indices.append(i)
    return indices

In [29]:
def create_token_embeddings(tokenized_text):
    """Convert the model into a more usable format: a tensor of size [<token_count>, <layer_count>, <feature_count>]."""
    input_ids = torch.tensor(tokenized_text).unsqueeze(0)  # Batch size 1
    with torch.no_grad():
        outputs = model(input_ids, masked_lm_labels=input_ids)
        encoded_layers = outputs[2]
        token_embeddings = torch.stack(encoded_layers, dim=0)  # Concatenate the tensors for all layers.
        token_embeddings = torch.squeeze(token_embeddings, dim=1)  # Remove the "batches" dimension
        token_embeddings = token_embeddings.permute(1,0,2)  # Rearrange the model dimensions.
        print(f'Size of token embeddings is {token_embeddings.size()}')
        return token_embeddings

In [40]:
def preview_token_embedding(tokenized_text, layer, index, index_list, tokenizer):
    """Print the first 5 feature values from a model layer for tokens at specific line indices."""
    v_index = index % len(tokenized_text[1:-1])
    print(f'{tokenizer.decode(tokenized_text[v_index + 1]).strip()} at index {index_list[index]}: ', \
          f'{layer[index_list[index]][:5].tolist()}')

In [23]:
def sum_last_four_token_vecs(token_embeddings):
    """Sum the last 4 layers' features and return the resulting vector."""
    token_vecs_sum_last_four = []

    # For each token in the sentence, sum the last 4 layers of the model.
    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)

        # Use `sum_vec` to represent `token`.
        token_vecs_sum_last_four.append(sum_vec)

    print ('Shape of summed layers is: %d x %d' % (len(token_vecs_sum_last_four), len(token_vecs_sum_last_four[0])))
    return token_vecs_sum_last_four

In [38]:
def get_layer_token_vecs(token_embeddings, layer_number):
    """Return a single layer of the model."""
    token_vecs_layer = []

    # For each token in the sentence...
    for token in token_embeddings:
        layer_vec = token[layer_number]

        # Use `layer_vec` to represent `token`.
        token_vecs_layer.append(layer_vec)

    print ('Shape of summed layers is: %d x %d' % (len(token_vecs_layer), len(token_vecs_layer[0])))
    return token_vecs_layer

In [8]:
def write_embedding(embeddings_file, vocab_word, contextual_embedding):
    """Save an embedding to an output file."""
    try:
        with open(embeddings_file, 'a') as f:
            f.write(vocab_word)
            for value in contextual_embedding[0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {vocab_word}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

In [44]:
def write_line_count(count_file, vocab_word, line_count):
    try:
        with open(count_file, 'a') as counts:
            counts.write(vocab_word + ', ' + str(line_count) + '\n')
        print(f'Saved the count of sentences used to create {vocab_word} embedding.')
    except:
        print('Wha?! Could not write the sentence count.')